In [ ]:
import os
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

# 데이터 디렉토리 및 클래스 레이블 설정
data_dir = '/content/drive/MyDrive/1. 회전기기 고장진단 분야 데이터/Data(프로젝트 데이터)'
classes = {
    'Normal': 0,
    'Inner Race': 1,
    'Misalignment': 2,
    'Outer Race': 3,
    'Roller': 4
}

data = []
labels = []

# 파일에서 데이터 로드하여 data와 labels 리스트에 추가
for class_name, class_label in classes.items():
    class_dir = os.path.join(data_dir, class_name)
    for file in os.listdir(class_dir):
        if file.endswith('.mat'):
            mat_data = sio.loadmat(os.path.join(class_dir, file))
            signals = mat_data['signals'][0]
            data.append(signals)
            labels.append(class_label)

data = np.array(data)
labels = np.array(labels)

# 주파수 도메인 특징 추출 함수 정의
def extract_frequency_features(data):
    frequency_features = []
    for signal in data:
        # FFT를 사용하여 주파수 영역으로 변환
        fft_result = np.fft.fft(signal)
        # 주파수 영역에서 관심 있는 주파수 성분 추출
        freq_range = slice(0, 1000)
        freq_features = np.abs(fft_result[freq_range])
        frequency_features.append(freq_features)
    return np.array(frequency_features)

# 주파수 도메인 특징 추출
frequency_features = extract_frequency_features(data)

# 데이터 분할 (학습 데이터와 테스트 데이터로 분리)
X_train, X_test, y_train, y_test = train_test_split(frequency_features, labels, test_size=0.2, random_state=42)

# 데이터 스케일링 (StandardScaler 사용)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_scaled = scaler.transform(frequency_features)

# SVM 모델 생성
svm_model = SVC(kernel='linear', C=1.0)

# SVM 모델 학습
svm_model.fit(X_scaled, labels)

In [ ]:
# Roller 데이터 디렉토리 설정
roller_dir = os.path.join() # Test Data 경로 입력

# 미래 예측할 데이터 로드하여 frequency domain 특징 추출
roller_data = []
roller_labels = []

for file in os.listdir(roller_dir):
    if file.endswith('.mat'):
        mat_data = sio.loadmat(os.path.join(roller_dir, file))
        signals = mat_data['signals'][0]
        roller_data.append(signals)

# 주파수 도메인 특징 추출
roller_frequency_features = extract_frequency_features(roller_data)

# 데이터 스케일링
roller_frequency_features_scaled = scaler.transform(roller_frequency_features)

# SVM 모델로 라벨 예측
predicted_labels = svm_model.predict(roller_frequency_features_scaled)

# 예측된 라벨 출력
print(predicted_labels)
